In [1]:
import os

In [2]:
%pwd

'e:\\END TO END ML PROJECT\\Text-Summarizer\\research'

In [3]:
os.chdir('E:\END TO END ML PROJECT\\Text-Summarizer')

In [4]:
%pwd

'E:\\END TO END ML PROJECT\\Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [8]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

d:\Applications\ANACONDA\ANACONDA_NAVIGATOR\envs\textS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        # Loading data in smaller chunks
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        train_dataset = dataset_samsum_pt["test"].shuffle(seed=42).select(range(5))
        eval_dataset = dataset_samsum_pt["validation"]

        # Reducing batch size and gradient accumulation
        per_device_train_batch_size = 2
        gradient_accumulation_steps = 8
        effective_batch_size = per_device_train_batch_size * gradient_accumulation_steps

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=1,
            warmup_steps=500,
            per_device_train_batch_size=per_device_train_batch_size,
            per_device_eval_batch_size=8,
            weight_decay=0.01,
            logging_steps=10,
            evaluation_strategy='steps',
            eval_steps=500,
            save_steps=1e6,
            gradient_accumulation_steps=gradient_accumulation_steps,
            dataloader_num_workers=4  # Adjust based on your system capabilities
        )

        trainer = Trainer(
            model=model_pegasus,
            args=trainer_args,
            tokenizer=tokenizer,
            data_collator=seq2seq_data_collator,A
            train_dataset=train_dataset,
            eval_dataset=eval_dataset
        )

        trainer.train()

        # Save model and tokenizer
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))


In [12]:
%pip install --upgrade accelerate
%pip uninstall -y transformers accelerate
%pip install transformers accelerate


Found existing installation: transformers 4.29.2
Uninstalling transformers-4.29.2:
  Successfully uninstalled transformers-4.29.2
Found existing installation: accelerate 0.19.0
Uninstalling accelerate-0.19.0:
  Successfully uninstalled accelerate-0.19.0
Note: you may need to restart the kernel to use updated packages.
  Using cached transformers-4.29.2-py3-none-any.whl (7.1 MB)
  Using cached accelerate-0.19.0-py3-none-any.whl (219 kB)
Note: you may need to restart the kernel to use updated packages.


In [10]:

try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2023-06-02 05:25:52,857 : INFO :  common : yaml file: config\config.yaml loaded successfully]
[2023-06-02 05:25:52,876 : INFO :  common : yaml file: params.yaml loaded successfully]
[2023-06-02 05:25:52,892 : INFO :  common : creating directory at : artifacts]
[2023-06-02 05:25:52,894 : INFO :  common : creating directory at : artifacts/model_trainer]
[2023-06-02 05:27:08,584 : WARNING :  arrow_dataset : Loading cached shuffled indices for dataset at E:\END TO END ML PROJECT\Text-Summarizer\artifacts\data_transformation\samsum_dataset\test\cache-58068f81d0dc399d.arrow]


d:\Applications\ANACONDA\ANACONDA_NAVIGATOR\envs\textS\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 1/1 [05:37<00:00, 337.12s/it]


{'train_runtime': 338.5898, 'train_samples_per_second': 0.015, 'train_steps_per_second': 0.003, 'train_loss': 1.535412073135376, 'epoch': 1.0}
